In [1]:
import pandas as pd
import os
import numpy as np
from copy import deepcopy

In [2]:
st = pd.HDFStore(os.path.expanduser("~/all.h5"), mode='r')

In [3]:
metadata_df = pd.read_csv("metadata/metadata.csv",index_col=0)

In [4]:
feeds = {'use':'aggregate',
        'air1':'hvac',
         'clotheswasher1':'wm',
         'dishwasher1':'dw',
         'microwave1':'mw',
         'refrigerator1':'fridge',
         'oven1':'oven',
        'car1':'ec',
         'waterheater1':'wh',
         'drye1':'dr'
        }

In [5]:
cities = ['Austin','Houston','San Diego','Boulder']

In [6]:
years = {
    2014:{'start':'1-1-2014', 'end':'12-31-2014', 'start_month':1, 'end_month':12, 'months':1},
    2015:{'start':'1-1-2015', 'end':'12-31-2015', 'start_month':1, 'end_month':12, 'months':1},
    2016:{'start':'1-1-2016', 'end':'12-31-2016', 'start_month':1, 'end_month':12, 'months':1}
    }

In [74]:
out_overall = {}
for year, year_dict in years.iteritems(): 
    out_overall[year] = {}
    to_ignore = []
    for city in cities[:]:
        print city, year
        city_data = metadata_df[metadata_df['city'] == city]
        city_homes = city_data.index.values.astype('int')
        out = {}
        for home in city_homes[:]:
            
            if home in to_ignore:
                continue
            try:
                data_df = st['/%d' %home][year_dict['start']:year_dict['end']]
                data_df_resampled = data_df.resample("1M", how="sum")
                df_res_kwh = data_df_resampled.div(1000)
                df_res_kwh.index = df_res_kwh.index.month
                # Find months where we have less than 28 days worth data and ignore those months
                day_res = data_df.resample("1D", how="sum")
                a = pd.value_counts(day_res.index.month)
                lt_28_days = a[a<28].index.values
                
                for m in lt_28_days:
                    df_res_kwh.loc[m] = np.NAN
                temp = {}
                for feed_name, feed_value in feeds.iteritems():
                    temp[feed_value] = []
                    for month in range(year_dict['start_month'], year_dict['end_month']+1):


                        try:
                            temp[feed_value].append(df_res_kwh.loc[month, feed_name])
                        except:
                            temp[feed_value].append(np.NAN)
                out[home] = np.hstack(temp.values())




            except Exception, e:
                pass
        appliances = temp.keys()
        col_headers = []
        for appliance in appliances:
            for month in range(1, 13):
                col_headers.append(appliance+"_"+str(month))
        df = pd.DataFrame(out).T
        df.columns = col_headers

        # Read metadata
        mdf = pd.read_csv("metadata/%s_md.csv" %city.replace(" ",""), index_col=0)
        for column in mdf.columns:
            df[column] = mdf[column]

        city = city.replace(" ","")
        out_overall[year][city] =df

        tdf = df[['aggregate_'+str(month) for month in range(1, 13)]].dropna().copy()
        full_agg = {}
     

        for home in df.index:
            if home in tdf.index:
                full_agg[home] = 1
            

            else:
                full_agg[home] = 0
     

Austin 2016


/Users/nipunbatra/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:16: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  app.launch_new_instance()
/Users/nipunbatra/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:20: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()


Houston 2016
San Diego 2016
Boulder 2016
Austin 2014
Houston 2014
San Diego 2014
Boulder 2014
Austin 2015
Houston 2015
San Diego 2015
Boulder 2015


In [75]:
def remove_lt(out_overall):
    for region in ['SanDiego','Austin']:
        for year in [2014, 2015, 2016]:
            df = out_overall[year][region]
            to_remove_agg_lt_appliance = []
            for appliance in ['hvac','wm','dw','mw','fridge','oven']:
                for month in range(1, 13):
                    a=df.query('aggregate_%d<1.35*%s_%d' %(month, appliance, month))
                    if len(a):
                        for x in a.index.values:
                            if x not in to_remove_agg_lt_appliance:
                                to_remove_agg_lt_appliance.append(x)
            print to_remove_agg_lt_appliance, year, region
            out_overall[year][region] = out_overall[year][region].drop(to_remove_agg_lt_appliance)
            
    return out_overall

In [76]:
out_overall = remove_lt(out_overall)

[] 2014 SanDiego
[] 2015 SanDiego
[] 2016 SanDiego
[1714, 4800, 5109, 8967, 5786, 9737, 3916, 4864, 4352, 2965] 2014 Austin
[5357, 2401, 5109, 3531, 8142, 7024, 8600, 1069] 2015 Austin
[2365, 9971, 2575, 5784, 668] 2016 Austin


### Removing outliers based on aggregate

1. Remove homes where aggregate less than 150 units
2. Remove high energy using z-score test

In [78]:
def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [79]:
for region in ['Austin']:
    for year in [2014, 2015, 2016]:
        to_remove = []
        for month in range(1, 13):
            df = out_overall[year][region]
            # Dropping outlier
            d = df['aggregate_%d' %month].dropna()
            drop_outlier = d.index[is_outlier(d)].values
            for x in drop_outlier:
                if x not in to_remove:
                    to_remove.append(x)
            
            # Drop lt
            drop_lt = d[d<150].index
            for x in drop_lt:
                if x not in to_remove:
                    to_remove.append(x)
            
        print region, year, to_remove
        out_overall[year][region] =  out_overall[year][region].drop(to_remove)
            
            
            

Austin 2014 [59, 222, 1507, 1700, 2995, 4154, 5357, 5673, 8079, 8197, 9356, 9647, 9745, 9771, 9922, 1069, 9578, 4957, 6799, 739, 3519, 5552, 7013, 7901, 7016, 9484, 1854, 8589, 9931, 252, 3631, 7512, 9701, 1500, 8031, 3299, 5456, 491, 9935, 2129, 9248, 3268, 3789, 3500, 6691, 7510, 2458, 4000, 7638, 2510, 8669]
Austin 2015 [59, 77, 222, 1507, 1700, 1994, 3916, 4154, 5673, 6691, 6799, 7016, 7510, 7788, 7965, 8197, 8419, 8669, 9356, 9647, 9745, 9771, 9922, 9935, 9938, 7512, 6941, 739, 3299, 3510, 7638, 9578, 9803, 3126, 3401, 4957, 5786, 898, 7900, 9248, 645, 3268, 4674, 5817, 6324, 4505, 7764, 4296, 252, 5784, 936]
Austin 2016 [222, 1507, 1700, 3126, 3401, 5357, 6691, 6799, 7510, 8197, 9356, 9647, 9922, 9935, 9938, 252, 4375, 9701, 7016, 3893, 8092, 2018, 4957, 1310, 3268, 8467, 9631, 7719, 2638, 4590, 3916, 8730]


In [80]:
%matplotlib inline

In [82]:
import pickle

In [83]:
pickle.dump(out_overall, open('metadata/all_regions_years_austin_cleaned.pkl','w'))